In [11]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

ModuleNotFoundError: No module named 'keras'

In [ ]:
file = h5py.File('Fort.mat', 'r')
data = file.get('Fort')
Fort = np.array(data)
F = Fort
scaler = MinMaxScaler(feature_range=(0, 1))
F = scaler.fit_transform(F)
F_tr = F[:150]
F_test = F[144:]
plt.figure(figsize = (10, 5))
plt.plot(F, 'k')
plt.plot(np.r_[:150],F_tr, 'b')
plt.plot(np.r_[144:174],F_test, 'r')
plt.show()

In [ ]:
# Предобработка исходных данных в формат batches для обучения\валидации
from keras.preprocessing.sequence import TimeseriesGenerator
data_gen = TimeseriesGenerator(F_tr, F_tr, length=6, sampling_rate=1, batch_size=150)
batch_0 = data_gen[0]
x, y = batch_0 # вход и обучающий выход для сети
print(x.shape) # 144 точки обучения, прогноз 1 точки по 6 прошлым
xx=np.reshape(x, (x.shape[0], 1, x.shape[1]))
yy=y
print(xx.shape) # меняем местами размерности
print(yy.shape)

In [ ]:
import keras
import pydot
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot

In [ ]:
# Составляется сама модель прогнозной сети
from keras.utils import plot_model
model = Sequential() # слои соединены последовательно
model.add(LSTM(units=20, input_shape=(1, 6))) # 20 нейронов
model.add(Dense(units = 1)) # выход одномерный
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
plot_model(model, to_file='model.png') # рисунок полученной сети

In [ ]:
from keras.utils import plot_model
model = Sequential()
model.add(LSTM(units=20, return_sequences=True, input_shape=(1, 6)))
model.add(Dropout(0.2))
model.add(LSTM(units=20, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=20))
model.add(Dense(units = 1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
plot_model(model, to_file='model.png')

In [ ]:
model.fit(xx, yy, epochs = 100) # 100 эпох по 144 точки

In [7]:
trainPredict = model.predict(xx)
trainPredict = scaler.inverse_transform(trainPredict)
plt.figure(figsize = (10, 5))
plt.plot(Fort, 'k')
plt.plot(np.r_[6:150],trainPredict, 'b')
plt.show()

NameError: name 'model' is not defined

In [ ]:
data_gen = TimeseriesGenerator(F_test, F_test,
 length=6, sampling_rate=1,
 batch_size=150)
batch_0 = data_gen[0]
x, y = batch_0
xx=np.reshape(x, (x.shape[0], 1, x.shape[1]))
yy = y
print(xx.shape) # прогноз на 24 точки по 6 наблюдениям
print(yy.shape) #

In [ ]:
testPredict = model.predict(xx)
testPredict = scaler.inverse_transform(testPredict)
plt.figure(figsize = (10, 5))
plt.plot(Fort, 'k')
plt.plot(np.r_[150:174],testPredict, 'b')
plt.show()

### Самостоятельный подбор

In [ ]:
file = h5py.File('Fort.mat', 'r')
data = file.get('Fort')
Fort = np.array(data)
F = Fort
scaler = MinMaxScaler(feature_range=(0, 1))
F = scaler.fit_transform(F)
F_tr = F[:144]
F_test = F[114:]
plt.figure(figsize = (10, 5))
plt.plot(F, 'k')
plt.plot(np.r_[:144],F_tr, 'b')
plt.plot(np.r_[114:174],F_test, 'r')
plt.show()

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
data_gen = TimeseriesGenerator(F_tr, F_tr, length=30, sampling_rate=1, batch_size=144)
batch_0 = data_gen[0]
x, y = batch_0 # вход и обучающий выход для сети
print(x.shape) # 114 точки обучения, прогноз 1 точки по 30 прошлым
xx=np.reshape(x, (x.shape[0], 1, x.shape[1]))
yy=y
print(xx.shape) # меняем местами размерности
print(yy.shape)

In [ ]:
from keras.utils import plot_model
model = Sequential()
model.add(LSTM(units=30, return_sequences=True, input_shape=(1, 30)))
model.add(Dropout(0.2))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=30))
model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')
plot_model(model, to_file='model.png')

In [ ]:
# 200 эпох по 114 точек обучения
model.fit(xx, yy, epochs = 200)

In [ ]:
trainPredict = model.predict(xx)
trainPredict = scaler.inverse_transform(trainPredict)
plt.figure(figsize = (10, 5))
plt.plot(Fort, 'k')
plt.plot(np.r_[30:144],trainPredict, 'b')
plt.show()

In [ ]:
data_gen = TimeseriesGenerator(F_test, F_test,
 length=30, sampling_rate=1,
 batch_size=144)
batch_0 = data_gen[0]
x, y = batch_0
xx=np.reshape(x, (x.shape[0], 1, x.shape[1]))
yy = y
print(xx.shape) # прогноз на 30 точки по 30 наблюдениям
print(yy.shape) #

In [ ]:
testPredict = model.predict(xx)
testPredict = scaler.inverse_transform(testPredict)
plt.figure(figsize = (10, 5))
plt.plot(Fort, 'k')
plt.plot(np.r_[144:174],testPredict, 'b')
plt.show()

### Свой вариант:  ВР вида полином. тренд + 2 сезонный компоненты + шум

In [ ]:
import numpy as np
import numpy.random as rand
import matplotlib.pyplot as plt
import h5py
%matplotlib inline

N = 1024
t = np.linspace(0, 1, N) 
b0 = 30     
b1 = 20     
b2 = 10     
b3 = 5
f1 = 10 
f2 = 3*f1 
F_TS =  b0 + b1 * t + b2 * t * t + b3 * t * t * t + np.sin(2 * np.pi * f1 * t) + np.sin(2 * np.pi * f2 * t) + 0.2 * rand.randn(len(t))

In [ ]:
plt.figure(figsize=(10,5)) #plot size
plt.plot(t,F_TS)

In [ ]:
# Масштабирование
scaler = MinMaxScaler(feature_range=(0, 1))
F = scaler.fit_transform(F_TS.reshape(-1,1))
F_tr = F[:700]
F_test = F[600:]
plt.figure(figsize = (10, 5))
plt.plot(F, 'k')
plt.plot(np.r_[:700],F_tr, 'b')
plt.plot(np.r_[600:1024],F_test, 'r')
plt.show()

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
data_gen = TimeseriesGenerator(F_tr, F_tr, length=100, sampling_rate=1, batch_size=700)
batch_0 = data_gen[0]
x, y = batch_0 # вход и обучающий выход для сети
print(x.shape) # 600 точек обучения, прогноз 1 точки по 100 прошлым
xx=np.reshape(x, (x.shape[0], 1, x.shape[1]))
yy=y
print(xx.shape) # меняем местами размерности
print(yy.shape)

In [ ]:
from keras.utils import plot_model
model = Sequential()
model.add(LSTM(units=20, return_sequences=True, input_shape=(1, 100)))
model.add(Dropout(0.2))
model.add(LSTM(units=20, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=20))
model.add(Dense(units = 1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
plot_model(model, to_file='model.png')

In [ ]:
# 150 эпох по 600 точек
model.fit(xx, yy, epochs = 150)

In [ ]:
trainPredict = model.predict(xx)
trainPredict = scaler.inverse_transform(trainPredict)
plt.figure(figsize = (10, 5))
plt.plot(F_TS, 'k')
plt.plot(np.r_[100:700],trainPredict, 'b')
plt.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
F = scaler.fit_transform(F_TS.reshape(-1,1))
F_tr = F[:700]
F_test = F[600:]
# plt.figure(figsize = (10, 5))
# plt.plot(F, 'k')
# plt.plot(np.r_[:700],F_tr, 'b')
# plt.plot(np.r_[600:1024],F_test, 'r')
# plt.show()


data_gen = TimeseriesGenerator(F_test, F_test, length=100, sampling_rate=1, batch_size=700)
batch_0 = data_gen[0]
x, y = batch_0
print(x.shape)
xx=np.reshape(x, (x.shape[0], 1, x.shape[1]))
yy = y
print(xx.shape) # прогноз на 324 точки по 100 наблюдениям
print(yy.shape) #

In [ ]:
testPredict = model.predict(xx)
testPredict = scaler.inverse_transform(testPredict)
plt.figure(figsize = (10, 5))
plt.plot(F_TS, 'k')
plt.plot(np.r_[700:1024],testPredict, 'b')
plt.show()

### 50 нейронов, 50 точек

In [ ]:
import numpy as np
import numpy.random as rand
import matplotlib.pyplot as plt
import h5py
%matplotlib inline
N = 1024
t = np.linspace(0, 1, N) 
b0 = 30     
b1 = 20     
b2 = 10     
b3 = 5
f1 = 10 
f2 = 3*f1 
F_2 =  b0 + b1 * t + b2 * t * t + b3 * t * t * t + np.sin(2 * np.pi * f1 * t) + np.sin(2 * np.pi * f2 * t) + 0.2 * rand.randn(len(t))
plt.figure(figsize=(10,5)) #plot size
plt.plot(t,F_2)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
F = scaler.fit_transform(F_2.reshape(-1,1))
F_tr = F[:900]
F_test = F[850:]
plt.figure(figsize = (10, 5))
plt.plot(F, 'k')
plt.plot(np.r_[:900],F_tr, 'b')
plt.plot(np.r_[850:1024],F_test, 'r')
plt.show()

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
data_gen = TimeseriesGenerator(F_tr, F_tr, length=50, sampling_rate=1, batch_size=900)
batch_0 = data_gen[0]
x, y = batch_0 # вход и обучающий выход для сети
print(x.shape) # 850 точки обучения, прогноз 1 точки по 50 прошлым
xx=np.reshape(x, (x.shape[0], 1, x.shape[1]))
yy=y
print(xx.shape) # меняем местами размерности
print(yy.shape)

In [ ]:
from keras.utils import plot_model
model = Sequential()
model.add(LSTM(units=35, return_sequences=True, input_shape=(1, 50)))
model.add(Dropout(0.2))
model.add(LSTM(units=35, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=35))
model.add(Dense(units = 1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
plot_model(model, to_file='model.png')

In [ ]:
model.fit(xx, yy, epochs = 100)

In [ ]:
trainPredict = model.predict(xx)
trainPredict = scaler.inverse_transform(trainPredict)
plt.figure(figsize = (10, 5))
plt.plot(F_2, 'k')
plt.plot(np.r_[50:900],trainPredict, 'b')
plt.show()

In [ ]:
data_gen = TimeseriesGenerator(F_test, F_test, length=50, sampling_rate=1, batch_size=900)
batch_0 = data_gen[0]
x, y = batch_0
xx=np.reshape(x, (x.shape[0], 1, x.shape[1]))
yy = y
print(xx.shape) # прогноз на 124 точки по 50 наблюдениям
print(yy.shape) #

In [ ]:
testPredict = model.predict(xx)
testPredict = scaler.inverse_transform(testPredict)
plt.figure(figsize = (10, 5))
plt.plot(F_2, 'k')
plt.plot(np.r_[900:1024],testPredict, 'b')
plt.show()